In [1]:
"""
Interactive Map with Dynamic Legends, Updated Data and Plot Integration from Google Drive,
Modified Layer 1 Tooltip & Popup with Google Drive Plot Dropdown.

This script:
  1. Downloads a CSV file from Google Drive and validates required columns.
  2. Authenticates with Google Drive via a service account to list image files from a specified folder.
  3. Sets up color mappings:
       - Layer 1: Linear colormap (yellow to green) for the "mean" field.
       - Layer 2: Distinct colors for bioregions.
       - Layer 3: Continuous red–yellow–green colormap for Mann-Kendall trend values.
  4. Creates three overlay layers:
       - Time Series Characteristics (Layer 1): Markers with tooltip showing Site ID and Mean PV;
         a popup includes detailed time series statistics, change point information, and a dropdown for plots
         whose images are served from Google Drive.
       - Bioregion (Layer 2): Markers colored by bioregion.
       - Mann-Kendall Trend Visualization (Layer 3): Markers colored by trend value.
  5. Adds dynamic legends that switch with the active overlay.
  6. Enforces that one base layer is always active.
  
Dependencies:
  - folium, pandas, branca, matplotlib, numpy, scipy, jinja2, gdown,
    google-api-python-client, google-auth, distinctipy (if needed)
"""

import folium
import pandas as pd
import branca.colormap as cm
import time
import matplotlib.colors as mcolors
import logging
import numpy as np
import math
import gdown

# Google Drive API imports
from googleapiclient.discovery import build
from google.oauth2 import service_account

from branca.element import Element

# -----------------------------------------------------------------------------
# 0. CONFIGURATION AND GOOGLE DRIVE AUTHENTICATION
# -----------------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
start_time = time.time()

# -------------------------------
# Google Drive: Image Files
# -------------------------------
# Folder ID in Google Drive containing the PNG images (plots)
FOLDER_ID = "1c92xgVH9N2LYEc1ucA5wiZ15pTE35SPT"
# https://drive.google.com/drive/folders/1c92xgVH9N2LYEc1ucA5wiZ15pTE35SPT?usp=sharing

# Authenticate using the service account JSON
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]
SERVICE_ACCOUNT_FILE = "driveimageaccess-450000-35cf361f9db1.json"  # Ensure this file is in your working directory

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
drive_service = build("drive", "v3", credentials=creds)

def list_drive_files():
    """
    List all PNG files in the designated Google Drive folder and return a
    dictionary mapping file names to a Google Drive thumbnail URL.
    """
    query = f"'{FOLDER_ID}' in parents and mimeType='image/png'"
    all_files = []
    page_token = None

    while True:
        results = drive_service.files().list(
            q=query,
            fields="nextPageToken, files(id, name)",
            pageToken=page_token,
            pageSize=1000  # maximum allowed page size
        ).execute()

        all_files.extend(results.get("files", []))
        page_token = results.get("nextPageToken")
        if not page_token:
            break

    return {
        file["name"]: f"https://drive.google.com/thumbnail?id={file['id']}&sz=w1000"
        for file in all_files
    }

logging.info("Fetching image links from Google Drive...")
image_links = list_drive_files()


# -------------------------------
# Google Drive: CSV Data File
# -------------------------------
# Google Drive file ID for the CSV data file
csv_file_id = "1KZMS4RrWMAudAs0_RKBfPOGyHZk-QUVk"
# https://drive.google.com/file/d/1KZMS4RrWMAudAs0_RKBfPOGyHZk-QUVk/view?usp=sharing

data_file = "site_data.csv"
logging.info("Downloading CSV data file from Google Drive...")
gdown.download(f"https://drive.google.com/uc?id={csv_file_id}", data_file, quiet=False)

# -----------------------------------------------------------------------------
# 1. LOAD AND VALIDATE DATA
# -----------------------------------------------------------------------------
logging.info("Step 1: Loading and validating data...")
df = pd.read_csv(data_file)

# Required columns (must match your CSV file)
required_columns = [
    "site_name", "latitude", "longitude", "bioregion_name", "mean",
    "MK trend_value", "min", "max", "MK_trend", "MK_p_value", 
    "cp_t", "cp_pr_t", "cp_s", "cp_pr_s"
]
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing columns in the data file: {missing_columns}")

# -----------------------------------------------------------------------------
# 2. SET UP COLOR MAPPING
# -----------------------------------------------------------------------------
logging.info("Step 2: Setting up color mapping...")

# ----- Layer 1: Linear colormap (yellow to green) for "mean" values -----
colormap_mean = cm.LinearColormap(
    colors=["yellow", "green"],
    vmin=df['mean'].min(),
    vmax=df['mean'].max()
)
colormap_mean.caption = "Mean FVC-PV Fraction"

# ----- Layer 3: Continuous colormap for Mann-Kendall Trend values -----
colormap_trend = cm.LinearColormap(
    colors=["red", "yellow", "green"],
    vmin=df['MK trend_value'].min(),
    vmax=df['MK trend_value'].max()
)
colormap_trend.caption = "Mann-Kendall Trend Value"

# ----- Layer 2: Distinct color mapping for Bioregions -----
bioregion_names = df['bioregion_name'].unique()

def generate_distinct_colors(n):
    """
    Generate n visually distinct colors using several matplotlib palettes.
    If additional colors are needed, the 'distinctipy' library is used.
    """
    palettes = [
        list(mcolors.TABLEAU_COLORS.values()),
        list(mcolors.XKCD_COLORS.values())[::15],  # subsample for diversity
        list(mcolors.CSS4_COLORS.values())[::10],
        list(mcolors.BASE_COLORS.values()),
    ]
    base_colors = []
    seen = set()
    for palette in palettes:
        for c in palette:
            if c not in seen:
                base_colors.append(c)
                seen.add(c)
    rng = np.random.default_rng(seed=42)
    rng.shuffle(base_colors)
    if n > len(base_colors):
        try:
            from distinctipy import distinctipy
        except ImportError:
            raise ImportError("Please install 'distinctipy' to generate additional distinct colors.")
        extra_colors = distinctipy.get_colors(n - len(base_colors))
        base_colors += [distinctipy.get_hex(c) for c in extra_colors]
    return base_colors[:n]

distinct_colors = generate_distinct_colors(len(bioregion_names))
bioregion_colors = {bioregion: color for bioregion, color in zip(bioregion_names, distinct_colors)}

# -----------------------------------------------------------------------------
# 3. CREATE THE MAP AND ADD BASE LAYERS
# -----------------------------------------------------------------------------
logging.info("Step 3: Generating map with base layers...")

m = folium.Map(
    location=[df["latitude"].mean(), df["longitude"].mean()],
    zoom_start=6,
    tiles=None
)

# Add base layers.
folium.TileLayer(
    tiles='https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    name="ESRI Satellite",
    attr="ESRI Satellite tiles",
    show=False
).add_to(m)
folium.TileLayer(
    tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    name="OpenStreetMap",
    attr="OpenStreetMap contributors",
    show=True
).add_to(m)

# Create overlay layers.
time_series_layer = folium.FeatureGroup(name="Time Series Characteristics", show=True)  # Layer 1
bioregion_layer = folium.FeatureGroup(name="Bioregion", show=False)                    # Layer 2
trend_layer = folium.FeatureGroup(name="Mann-Kendall Trend Visualization", show=False)  # Layer 3

time_series_layer.add_to(m)
bioregion_layer.add_to(m)
trend_layer.add_to(m)

# -----------------------------------------------------------------------------
# 4. ADD MARKERS WITH INTERACTIVE TOOLTIP AND POPUP (Layer 1 Updated)
# -----------------------------------------------------------------------------
logging.info("Step 4: Adding markers with interactive tooltips/popups...")

# Mapping for plot file suffixes and display names for Layer 1 popup.
plot_types = {
    "timeseries": "Time Series Plot",
    "bayesian": "Time Series Decomposition",
    "pv_spiral": "Spiral Plot",
    "polar": "Polar Plot",   
}

for _, row in df.iterrows():
    if pd.isnull(row["latitude"]) or pd.isnull(row["longitude"]):
        continue

    site_name = row["site_name"]
    
    # ----- Layer 1: Time Series Characteristics -----
    time_series_color = colormap_mean(row["mean"])
    tooltip_text = f"Site ID: {site_name}<br>Mean PV: {row['mean']:.2f}"

    # Build change point table (as HTML)
    change_points_table = pd.DataFrame({
        "Description": [
            "Changepoints (trend)", "Changepoints probability (trend)",
            "Changepoints (season)", "Changepoints probability (season)"
        ],
        "Value": [
            f"{row['cp_t']}", f"{row['cp_pr_t']}",
            f"{row['cp_s']}", f"{row['cp_pr_s']}"
        ]
    }).to_html(index=False, classes="table table-bordered", escape=False)
    change_points_table = change_points_table.replace("<th>", "<th style='text-align: left;'>")
    
    # Generate dropdown options for plots using Google Drive image links.
    plot_dropdown = "".join([
        f'<option value="{image_links.get(f"{site_name}_{suffix}.png", "#")}">{display_name}</option>'
        for suffix, display_name in plot_types.items()
    ])
    
    # Build popup content for Layer 1.
    time_series_popup_content = f"""
        <div style="width: 750px; max-height: 800px; overflow-y: auto;">
            <p><b>Site ID:</b> {site_name}</p>
            <p><b>Min:</b> {row['min']:.3f}, <b>Max:</b> {row['max']:.3f}</p>
            <p><b>Mean:</b> {row['mean']:.2f}</p>
            <p><b>Trend and Seasonality Change Points:</b></p>
            {change_points_table}
            <p><b>Time Series Plots:</b></p>
            <select id="plotSelect_{site_name}" onchange="updatePlot('{site_name}', this.value);">
                <option value="" selected disabled>Select a plot</option>
                {plot_dropdown}
            </select>
            <div style="text-align: center; margin-top: 10px;">
                <img id="plotImage_{site_name}" src="" style="width: 700px; max-width: 100%; height: auto; display: none;" alt="No Image Selected">
                <p id="imageError_{site_name}" style="color: red; display: none;">⚠️ No image available ⚠️</p>
            </div>
        </div>
    """

    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=8,
        fill=True,
        fill_opacity=0.9,
        color="grey",
        weight=1.5,
        fill_color=time_series_color,
        tooltip=tooltip_text,
        popup=folium.Popup(time_series_popup_content, max_width=800)
    ).add_to(time_series_layer)

    # ----- Layer 2: Bioregion -----
    bioregion_color = bioregion_colors[row['bioregion_name']]
    bioregion_popup_content = f"""
        <b>Site ID:</b> {site_name}<br>
        <b>Bioregion:</b> {row['bioregion_name']}
    """
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=8,
        fill=True,
        fill_opacity=0.9,
        color="grey",
        weight=1.5,
        fill_color=bioregion_color,
        tooltip=bioregion_popup_content
    ).add_to(bioregion_layer)

    # # ----- Layer 3: Mann-Kendall Trend Visualization ----
    # trend_color = colormap_trend(row['MK trend_value'])
    # trend_popup_content = f"""
    #     <b>Mann-Kendall Trend Statistics</b><br><br>
    #     <b>Site ID:</b> {site_name}<br>
    #     <b>Trend Value:</b> {row['MK trend_value']:.3f}<br>
    #     <b>Trend:</b> {row['MK_trend']}<br>
    #     <b>P-Value:</b> {row['MK_p_value']:.2e}
    # """

    # folium.CircleMarker(
    #     location=[row["latitude"], row["longitude"]],
    #     radius=8,
    #     fill=True,
    #     fill_opacity=0.9,
    #     color="grey",
    #     weight=1.5,
    #     fill_color=trend_color,
    #     tooltip=trend_popup_content
    # ).add_to(trend_layer)



    # ----- Layer 3: Mann-Kendall Trend Visualization ----
    trend_color = colormap_trend(row['MK trend_value'])
    tooltip_text_trend = (
        f"<b>Mann-Kendall Trend Statistics</b><br>"
        f"Site ID: {site_name}<br>"
        f"Trend Value: {row['MK trend_value']:.3f}<br>"
        f"Trend: {row['MK_trend']}<br>"
        f"P-Value: {row['MK_p_value']:.2e}"
    )


    # Build dropdown options for Layer 3: include only the "timeseries" plot option.
    trend_plot_dropdown = "".join([
        f'<option value="{image_links.get(site_name + "_" + "timeseries" + ".png", "#")}">{plot_types["timeseries"]}</option>'
    ])

    # Build popup content for Layer 3 – similar to Layer 1 but with trend statistics.
    trend_popup_content = f"""
        <div style="width: 750px; max-height: 800px; overflow-y: auto;">
            <p><b>Mann-Kendall Trend Statistics</b></p>
            <p><b>Site ID:</b> {site_name}</p>
            <p><b>Trend Value:</b> {row['MK trend_value']:.3f}</p>
            <p><b>Trend:</b> {row['MK_trend']}</p>
            <p><b>P-Value:</b> {row['MK_p_value']:.2e}</p>
            <p><b>Time Series Plot:</b></p>
            <select id="plotSelect_trend_{site_name}" onchange="updatePlot('{site_name}', this.value);">
                <option value="" selected disabled>Select a plot</option>
                {trend_plot_dropdown}
            </select>
            <div style="text-align: center; margin-top: 10px;">
                <img id="plotImage_{site_name}" src="" style="width: 700px; max-width: 100%; height: auto; display: none;" alt="No Image Selected">
                <p id="imageError_{site_name}" style="color: red; display: none;">⚠️ No image available ⚠️</p>
            </div>
        </div>
    """
    
    # Use a simple tooltip for Layer 3 so that the detailed content (with drop-down) is shown only on click.
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=8,
        fill=True,
        fill_opacity=0.9,
        color="grey",
        weight=1.5,
        fill_color=trend_color,
        tooltip=tooltip_text_trend,
        popup=folium.Popup(trend_popup_content, max_width=800)
    ).add_to(trend_layer)

# -----------------------------------------------------------------------------
# 5. CREATE DYNAMIC LEGENDS
# -----------------------------------------------------------------------------
logging.info("Step 5: Adding dynamic legends...")

min_mean = df['mean'].min()
max_mean = df['mean'].max()
avg_mean = df['mean'].mean()
min_trend = df['MK trend_value'].min()
max_trend = df['MK trend_value'].max()

# Legend for Layer 1: Time Series Characteristics
legend_layer1 = f"""
<div id="legend-layer1" style="
    position: fixed;
    top: 160px;
    right: 20px;
    z-index: 9999;
    display: none;
    font-family: Arial, sans-serif;
    font-size: 12px;
    width: 220px;">
  <div style="font-weight: bold; margin-bottom: 5px;">Mean FVC-PV Fraction</div>
  <div style="height: 20px; background: linear-gradient(to right, yellow, green);"></div>
  <div style="display: flex; justify-content: space-between; margin-top: 2px;">
      <span>{min_mean:.2f}</span>
      <span>{max_mean:.2f}</span>
  </div>
  <div style="text-align: center; margin-top: 2px;">Avg: {avg_mean:.2f}</div>
</div>
"""

# Legend for Layer 2: Bioregion (3 columns, increased font size)
bioregions_sorted = sorted(bioregion_colors.items(), key=lambda x: x[0])
n_bioregions = len(bioregions_sorted)
col_size = math.ceil(n_bioregions / 3)
col1_list = bioregions_sorted[0:col_size]
col2_list = bioregions_sorted[col_size:2*col_size]
col3_list = bioregions_sorted[2*col_size:]

def build_bioregion_column(col_items):
    html = ""
    for region, color in col_items:
        html += f"<p style='margin: 2px 0; font-size: 16px;'><span style='color:{color};'>&#9679;</span> {region}</p>"
    return html

legend_layer2 = f"""
<div id="legend-layer2" style="
    position: fixed;
    top: 160px;
    right: 20px;
    z-index: 9999;
    display: none;
    font-family: Arial, sans-serif;
    font-size: 16px;
    width: 240px;">
  <div style="font-weight: bold; margin-bottom: 5px;">Bioregion</div>
  <table style="width:100%;">
    <tr>
      <td valign="top" style="width:33%;">{build_bioregion_column(col1_list)}</td>
      <td valign="top" style="width:33%;">{build_bioregion_column(col2_list)}</td>
      <td valign="top" style="width:34%;">{build_bioregion_column(col3_list)}</td>
    </tr>
  </table>
</div>
"""

# Legend for Layer 3: Mann-Kendall Trend Visualization
legend_layer3 = f"""
<div id="legend-layer3" style="
    position: fixed;
    top: 160px;
    right: 20px;
    z-index: 9999;
    display: none;
    font-family: Arial, sans-serif;
    font-size: 12px;
    width: 240px;">
  <div style="font-weight: bold; margin-bottom: 5px;">Mann-Kendall Trend Value</div>
  <div style="height: 20px; background: linear-gradient(to right, red, yellow, green);"></div>
  <div style="display: flex; justify-content: space-between; margin-top: 2px;">
      <span>{min_trend:.2f}</span>
      <span>{max_trend:.2f}</span>
  </div>
</div>
"""

# Inject legends into the map.
m.get_root().html.add_child(Element(legend_layer1))
m.get_root().html.add_child(Element(legend_layer2))
m.get_root().html.add_child(Element(legend_layer3))

# -----------------------------------------------------------------------------
# 6. ADD JAVASCRIPT FOR DYNAMIC LEGEND SWITCHING, LAYER CONTROL, AND UPDATE PLOT
# -----------------------------------------------------------------------------
js_script = """
<script>
function updateLegend(activeLayerName) {
    // Hide all legends.
    document.getElementById('legend-layer1').style.display = 'none';
    document.getElementById('legend-layer2').style.display = 'none';
    document.getElementById('legend-layer3').style.display = 'none';
    
    // Display legend based on active overlay.
    if (activeLayerName.trim() === 'Time Series Characteristics') {
        document.getElementById('legend-layer1').style.display = 'block';
    } else if (activeLayerName.trim() === 'Bioregion') {
        document.getElementById('legend-layer2').style.display = 'block';
    } else if (activeLayerName.trim() === 'Mann-Kendall Trend Visualization') {
        document.getElementById('legend-layer3').style.display = 'block';
    }
}

function enforceSingleLayerSelection() {
    // Attach listeners to overlay checkboxes.
    var overlayCheckboxes = document.querySelectorAll(".leaflet-control-layers-overlays input[type='checkbox']");
    overlayCheckboxes.forEach(function(checkbox) {
        checkbox.addEventListener('change', function() {
            setTimeout(function() {
                overlayCheckboxes.forEach(function(other) {
                    if (other !== checkbox) { other.checked = false; }
                });
                var label = checkbox.nextElementSibling ? checkbox.nextElementSibling.innerText : "";
                updateLegend(label);
            }, 300);
        });
    });
    // Ensure one base layer is always selected.
    var baseLayerRadios = document.querySelectorAll(".leaflet-control-layers-base input[type='radio']");
    baseLayerRadios.forEach(function(radio) {
        radio.addEventListener('change', function() {
            setTimeout(function() {
                var selected = document.querySelector(".leaflet-control-layers-base input:checked");
                if (!selected && baseLayerRadios.length > 0) {
                    baseLayerRadios[0].checked = true;
                }
            }, 300);
        });
    });
}

// Function to update the plot image in the popup for Layer 1.
function updatePlot(site, value) {
    console.log("Site:", site, "Selected Value:", value);
    var img = document.getElementById("plotImage_" + site);
    var errorText = document.getElementById("imageError_" + site);
    if (value && value !== "#") {
        img.src = value;
        img.style.display = "block";
        errorText.style.display = "none";
    } else {
        img.style.display = "none";
        errorText.style.display = "block";
    }
}

document.addEventListener("DOMContentLoaded", function() {
    setTimeout(function() {
        enforceSingleLayerSelection();
        // Set default legend.
        updateLegend('Time Series Characteristics');
    }, 1000);
});
</script>
"""
m.get_root().html.add_child(Element(js_script))

# -----------------------------------------------------------------------------
# 7. ADD LAYER CONTROL AND SAVE THE MAP
# -----------------------------------------------------------------------------
folium.LayerControl(collapsed=False).add_to(m)

logging.info("Step 7: Saving the map...")
output_file = f"GDrive_map_with_dynamic_legends_{time.strftime('%Y%m%d_%H%M%S')}.html"
m.save(output_file)
logging.info(f"Map saved as '{output_file}'.")

print(f"Total Time Taken: {time.time() - start_time:.2f} seconds.")


2025-02-11 23:44:46,915 - file_cache is only supported with oauth2client<4.0.0
2025-02-11 23:44:46,947 - Fetching image links from Google Drive...
2025-02-11 23:44:56,430 - Downloading CSV data file from Google Drive...
Downloading...
From: https://drive.google.com/uc?id=1KZMS4RrWMAudAs0_RKBfPOGyHZk-QUVk
To: C:\Users\lamsalk\OneDrive - University of Tasmania\_TERN\ausplot_jupyternotebook\interactive_plot\site_data.csv
100%|███████████████████████████████████████████████████████████████████████████████| 253k/253k [00:00<00:00, 2.59MB/s]
2025-02-11 23:45:01,390 - Step 1: Loading and validating data...
2025-02-11 23:45:01,458 - Step 2: Setting up color mapping...
2025-02-11 23:45:01,467 - Step 3: Generating map with base layers...
2025-02-11 23:45:01,499 - Step 4: Adding markers with interactive tooltips/popups...
2025-02-11 23:45:03,814 - Step 5: Adding dynamic legends...
2025-02-11 23:45:03,821 - Step 7: Saving the map...
2025-02-11 23:45:11,692 - Map saved as 'GDrive_map_with_dynamic_l

Total Time Taken: 25.02 seconds.
